In [ ]:
import os
main_dir = os.path.dirname(os.path.dirname(os.getcwd()))
os.chdir(main_dir)

import pyvdirs.dirs as dirs
import sys
sys.path.insert(0, dirs.SYSTEM_HOME)
sys.path.insert(0, os.path.join(dirs.SYSTEM_HOME, "ToyExample"))

import builtins
import pickle
import torch
import numpy as np
import matplotlib.pyplot as plt

from ToyExample.toy_example import gt, mandala_score, plot_mandala_score

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [101]:
series = "30_TestEfficiency"
classes='A'
test_n_samples = 10*2**14
vmax = 10250 # 3600

gnet_path = os.path.join(dirs.MODELS_HOME, "ToyExample/00_PreTrained/Ref/iter0512.pkl")

device = torch.device('cuda')
full_tree = False
show_points = False

In [102]:
if full_tree: kwargs = {}
elif not full_tree and show_points: kwargs = dict(show_titles=False, full_tree=False, show_points=True)
else: kwargs = dict(show_titles=False, full_tree=False, show_points=False)

In [103]:
seeds = [0, 73, 231, 172, 357]
data = "> 30_TestEfficiency ACIDNonInverted_073 >>> Time >>> 1279 >>> 15.866415882110596\
> 30_TestEfficiency ACIDNonInverted_073 >>> Examples >>> 4095 >>> 3342336\
> 30_TestEfficiency ACIDNonInverted_231 >>> Time >>> 1279 >>> 15.861848334471384\
> 30_TestEfficiency ACIDNonInverted_231 >>> Examples >>> 4095 >>> 3342336\
> 30_TestEfficiency ACIDNonInverted_000 >>> Time >>> 1279 >>> 15.796777415275574\
> 30_TestEfficiency ACIDNonInverted_000 >>> Examples >>> 4095 >>> 3342336\
> 30_TestEfficiency ACIDNonInverted_172 >>> Time >>> 1279 >>> 15.885940523942311 \
> 30_TestEfficiency ACIDNonInverted_172 >>> Examples >>> 4095 >>> 3342336 \
> 30_TestEfficiency ACIDNonInverted_357 >>> Time >>> 1535 >>> 16.134254491329195 \
> 30_TestEfficiency ACIDNonInverted_357 >>> Examples >>> 4095 >>> 3342336\
> 30_TestEfficiency EarlyACIDNonInverted_073 >>> Time >>> 4095 >>> 17.272693932056427\
> 30_TestEfficiency EarlyACIDNonInverted_073 >>> Examples >>> 1023 >>> 3630144\
> 30_TestEfficiency EarlyACIDNonInverted_357 >>> Time >>> 3583 >>> 16.468802710374195\
> 30_TestEfficiency EarlyACIDNonInverted_357 >>> Examples >>> 1023 >>> 3003664 \
> 30_TestEfficiency EarlyACIDNonInverted_172 >>> Time >>> 3839 >>> 16.44512361685435 \
> 30_TestEfficiency EarlyACIDNonInverted_172 >>> Examples >>> 1023 >>> 3564544 \
> 30_TestEfficiency EarlyACIDNonInverted_231 >>> Time >>> 3583 >>> 16.695520043373108 \
> 30_TestEfficiency EarlyACIDNonInverted_231 >>> Examples >>> 1023 >>> 3020064 \
> 30_TestEfficiency EarlyACIDNonInverted_000 >>> Time >>> 4095 >>> 17.21331090927124 \
> 30_TestEfficiency EarlyACIDNonInverted_000 >>> Examples >>> 1023 >>> 3561264 \
> 30_TestEfficiency Random_000 >>> Time >>> 3839 >>> 16.457821106910707 \
> 30_TestEfficiency Random_000 >>> Examples >>> 4095 >>> 3354624 \
> 30_TestEfficiency Random_172 >>> Time >>> 3839 >>> 16.855511740843454 \
> 30_TestEfficiency Random_172 >>> Examples >>> 4095 >>> 3354624 \
> 30_TestEfficiency Random_357 >>> Time >>> 3839 >>> 16.96301788886388 \
> 30_TestEfficiency Random_357 >>> Examples >>> 4095 >>> 3354624 \
> 30_TestEfficiency Random_073 >>> Time >>> 4095 >>> 16.956689266363778 \
> 30_TestEfficiency Random_073 >>> Examples >>> 4095 >>> 3354624 \
> 30_TestEfficiency Random_231 >>> Time >>> 3839 >>> 16.97756971915563 \
> 30_TestEfficiency Random_231 >>> Examples >>> 4095 >>> 3354624\
> 30_TestEfficiency NoACID_357 >>> Time >>> 3839 >>> 16.64228993654251\
> 30_TestEfficiency NoACID_357 >>> Examples >>> 767 >>> 3145728\
> 30_TestEfficiency NoACID_000 >>> Time >>> 3839 >>> 17.218669092655183 \
> 30_TestEfficiency NoACID_000 >>> Examples >>> 767 >>> 3145728 \
> 30_TestEfficiency NoACID_073 >>> Time >>> 3839 >>> 16.55296248992284 \
> 30_TestEfficiency NoACID_073 >>> Examples >>> 767 >>> 3145728 \
> 30_TestEfficiency NoACID_172 >>> Time >>> 3839 >>> 16.540579922993977 \
> 30_TestEfficiency NoACID_172 >>> Examples >>> 767 >>> 3145728 \
> 30_TestEfficiency NoACID_231 >>> Time >>> 3839 >>> 16.666315853595734 \
> 30_TestEfficiency NoACID_231 >>> Examples >>> 767 >>> 3145728 "

In [104]:
data = [d for d in data.split("> 30_TestEfficiency") if d != ""]

In [105]:
data = [d.split(" ") for d in data]
data = [[d for d in dat if d not in ["",">>>"]] for dat in data]
data = [[*d[0].split("_"), *d[1:]] for d in data]

In [107]:
times = [float(dat[-1]) for dat in data if "Time" in dat]
print(np.mean(times), np.std(times))

16.573605743845302 0.4583550910060457


In [108]:
examples = [float(dat[-1]) for dat in data if "Examples" in dat]
print(np.mean(examples), np.std(examples))

3299656.0 166777.41602507216


In [109]:
sorted_data = {}
for d in data:
    if d[2] not in sorted_data.keys():
        sorted_data[ d[2] ] = {}
    if d[0] not in sorted_data[ d[2] ].keys():
        sorted_data[ d[2] ][ d[0] ] = []
    sorted_data[ d[2] ][ d[0] ].append( [d[1],d[-2],d[-1]] )

### Don't accumulate

In [ ]:
# Guided
gtd, gtcomps = gt(classes, device)
with builtins.open(gnet_path, "rb") as f:
    gnet = pickle.load(f).to(device)
for k in sorted_data.keys():
    for folder, dat in sorted_data[k].items():
        for d in dat:
            seed, epoch, time = d
            epoch = int(epoch)

            net_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", 
                                        series, folder, f"iter{epoch+1:04}.pkl")
            with builtins.open(net_filepath, "rb") as f:
                net = pickle.load(f).to(device)

            if full_tree:
                fig_filename = f"mandala_{k.lower()}_guided_{epoch+1:04}_"+folder+".png"
            elif not show_points:
                fig_filename = f"mandala_{k.lower()}_guided_{epoch+1:04}_"+folder+"_2.png"
            else:
                fig_filename = f"mandala_{k.lower()}_guided_{epoch+1:04}_"+folder+"_3.png"
            fig_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, fig_filename)

            mandala_score(net, gtd, guide=gnet, guidance_weight=3, 
                          n_samples=test_n_samples, plotting=True, 
                          save_fig=fig_filepath, show_stats=False, 
                          vmax=vmax, log_scale=True, **kwargs);
            plt.close()

In [ ]:
# Unguided
gtd, gtcomps = gt(classes, device)
with builtins.open(gnet_path, "rb") as f:
    gnet = pickle.load(f).to(device)
for k in sorted_data.keys():
    for folder, dat in sorted_data[k].items():
        for d in dat:
            seed, epoch, time = d
            epoch = int(epoch)

            net_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", 
                                        series, folder, f"iter{epoch+1:04}.pkl")
            with builtins.open(net_filepath, "rb") as f:
                net = pickle.load(f).to(device)

            if full_tree:
                fig_filename = f"mandala_{k.lower()}_{epoch+1:04}_"+folder+".png"
            elif not show_points:
                fig_filename = f"mandala_{k.lower()}_{epoch+1:04}_"+folder+"_2.png"
            else:
                fig_filename = f"mandala_{k.lower()}_{epoch+1:04}_"+folder+"_3.png"
            fig_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, fig_filename)

            mandala_score(net, gtd, n_samples=test_n_samples, plotting=True, 
                          save_fig=fig_filepath, show_stats=False, 
                          vmax=vmax, log_scale=True, **kwargs);
            plt.close()

### Accumulate grid

In [124]:
# Guided
gtd, gtcomps = gt(classes, device)
with builtins.open(gnet_path, "rb") as f:
    gnet = pickle.load(f).to(device)
for k in sorted_data.keys():
    for folder, dat in sorted_data[k].items():
        averaged_grid = []
        for d in dat:
            seed, epoch, time = d
            epoch = int(epoch)
            net_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", 
                                        series, f"{folder}_{seed:03}", 
                                        f"iter{epoch+1:04}.pkl")
            with builtins.open(net_filepath, "rb") as f:
                net = pickle.load(f).to(device)
            plotting_args = mandala_score(net, gtd, guide=gnet, guidance_weight=3, 
                                          n_samples=test_n_samples)[-1]
            averaged_grid.append( plotting_args["grid"])

        if full_tree:
            fig_filename = f"mandala_{k.lower()}_guided_{epoch+1:04}_"+folder+".jpg"
        elif not show_points:
            fig_filename = f"mandala_{k.lower()}_guided_{epoch+1:04}_"+folder+"_2.jpg"
        else:
            fig_filename = f"mandala_{k.lower()}_guided_{epoch+1:04}_"+folder+"_3.jpg"
        fig_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, fig_filename)
        
        del plotting_args["grid"]
        plotting_args["grid"] = np.mean(averaged_grid, axis=0)
        plot_mandala_score(**plotting_args, save_fig=fig_filepath, show_stats=False, 
                            vmax=vmax, log_scale=True, **kwargs)
        plt.close()

In [ ]:
# Unguided
gtd, gtcomps = gt(classes, device)
with builtins.open(gnet_path, "rb") as f:
    gnet = pickle.load(f).to(device)
for k in sorted_data.keys():
    for folder, dat in sorted_data[k].items():
        averaged_grid = []
        for d in dat:
            seed, epoch, time = d
            epoch = int(epoch)
            net_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", 
                                        series, f"{folder}_{seed:03}", 
                                        f"iter{epoch+1:04}.pkl")
            with builtins.open(net_filepath, "rb") as f:
                net = pickle.load(f).to(device)
            plotting_args = mandala_score(net, gtd, n_samples=test_n_samples)[-1]
            averaged_grid.append( plotting_args["grid"])

        if full_tree:
            fig_filename = f"mandala_{k.lower()}_{epoch+1:04}_"+folder+".jpg"
        elif not show_points:
            fig_filename = f"mandala_{k.lower()}_{epoch+1:04}_"+folder+"_2.jpg"
        else:
            fig_filename = f"mandala_{k.lower()}_{epoch+1:04}_"+folder+"_3.jpg"
        fig_filepath = os.path.join(dirs.MODELS_HOME, "ToyExample", series, fig_filename)
        
        del plotting_args["grid"]
        plotting_args["grid"] = np.mean(averaged_grid, axis=0)
        plot_mandala_score(**plotting_args, save_fig=fig_filepath, show_stats=False, 
                            vmax=vmax, log_scale=True, **kwargs)
        plt.close()